In [0]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

# 6. 합성공 신경망(Convolutional Neural Network)



합성공 신경망은 신경망 구조에서 convolution의 연산을 수행하여 동작합니다. 이는 이미지를 처리할때 효율적(localization, feature extraction)한다고 알려져 있습니다. 

기존 신경망과의 차이점은 convolution 연산을 위한 다음의 3가지 레이어로 구분되서 신경망이 구성된다는 점입니다.

* convolutional layer
* pooling layer
* fully-conntected layer

각 레이어들에 대해 살펴봅시다.

## 6.1 Convolutional layer


![](https://image.slidesharecdn.com/cs231-part2-170403153719/95/review-on-cs231-part2-4-638.jpg?cb=1491234054)

32x32x3의 이미지가 존재한다고 했을때 컨볼루션 레이어는 입력데이터의 일부분과 필터(kernel)를 컨볼루션합니다. 위 그림에서 필터는 5x5x3이고 마지막 3은 인풋이미지와 동일한 이미지의 채널을 의미합니다. 이때 컨볼류션의 결과는 1number입니다. 입력데이터와 필터의 값을 각각 곱하고 다 더한 결과입니다. 이 과정을 인풋데이터 전체를 거칠때까지 좌로, 아래로 이동하며 수행됩니다.

### 필터

그럼 필터는 
[그림]

니다.

또한 아웃풋 feature map의 채널은 사용되는 필터 수라는 것도 알수 있습니다.

인풋데이터에 필터에 대한 컨볼루션 연산을 한다는 것은 다음과 같습니다.

**필터의 개수는 아웃풋 feature map의 채널수가 됩니다.**

![](https://youngwonseo.github.io/public/deeplearning/convolution1.PNG)

![](https://youngwonseo.github.io/public/deeplearning/convolution2.PNG)

![](https://youngwonseo.github.io/public/deeplearning/convolution3.PNG)

![](https://youngwonseo.github.io/public/deeplearning/convolution4.PNG)

![](https://youngwonseo.github.io/public/deeplearning/convolution5.PNG)

실제 필터의 크기 및 수는 하이퍼파라미터입니다. 

그럼 3채널 데이터에 대한 convolution의 이해해봅시다. 인풋데이터의 각 채널별로 필터가 할당됩니다. 즉 3채널 인풋 데이터는 3채널 필터가 적용되죠,  그리고 필터를 컨볼루션하면 각 채널별로 한개의 값이 나옵니다. 즉 위 그림에서 RGB 순으로 필터를 적용하면 3개의 스칼라 값이 나오는데 결과 feature map은 3개의 숫자를 더한 결과입니다. **컨볼루션의 결과는 항상 1개의 숫자입니다.**


![](https://youngwonseo.github.io/public/deeplearning/convolution_3_3.PNG)

![](https://youngwonseo.github.io/public/deeplearning/convolution_3_2.gif)



### stride
스트라이드는 필터를 적용할때 움직일 크기 입니다.



### padding

이미지의 가장자리에 0를 추가해 output feature가 작아져 정보가 소멸되는것을 방지합니다. 텐서플로우에서는 패딩에 대한 옵션으로 SAME과 VALID를 제공하는데 SAME은 인웃과 아웃풋의 크기가 같도록 PAD를 추가합니다.

![](https://youngwonseo.github.io/public/deeplearning/max_pooling.png)


인풋데이터와 필터, 스트라이브, 패딩에 따라 결정된는 아웃풋 데이터의 크기는 다음과 같습니다.

$$ n_H = \lfloor \frac{n_{H_{prev}} - f + 2 \times pad}{stride} \rfloor +1 $$
$$ n_W = \lfloor \frac{n_{W_{prev}} - f + 2 \times pad}{stride} \rfloor +1 $$
$$ n_C = \text{number of filters used in the convolution}$$




### Relu

아웃풋 feature map에 ReLU 적용



## 6.2 Pooling layer

Max, Avarage
컨볼루션 레이어로 출련되는 피처맵을 샘플링(압축)하기 위해 풀링 레이어를 사용합니다. 

![](https://t1.daumcdn.net/cfile/tistory/2519E53757AB54E907)


풀링결과인 output feature맵의 channel수는 입력 feature map과 같습니다. 즉 풀링은 채널의 수를 변경시키지는 않습니다.

풀링도 필터(크기)와 stride가 필요
 
 
 
 
 
 
 인풋이 28 x 28 일때 stride 2, padding이 SAME이면 아웃풋은 14 x 14

## 6.3 소프트맥스

합성곱 신경망에서는 fully conntected layer를 통해 마지막에 소프트맥스 계층을 연결합니다. 이는 출력에 대한, 즉 classification에 대한 결과를 반환합니다. 만약 class가 10개라면 softmax layer의 뉴런은 10개가 존재하고 10개의 출력의 합이 1이되도록 아웃풋을 반환합니다.

In [0]:
batch_size = 128
test_size = 256

def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

def model(X, w, w2, w3, w4, w_o, p_keep_conv, p_keep_hidden):
    l1a = tf.nn.relu(tf.nn.conv2d(X, w,                       # l1a shape=(?, 28, 28, 32)
                        strides=[1, 1, 1, 1], padding='SAME'))
    l1 = tf.nn.max_pool(l1a, ksize=[1, 2, 2, 1],              # l1 shape=(?, 14, 14, 32)
                        strides=[1, 2, 2, 1], padding='SAME')
    l1 = tf.nn.dropout(l1, p_keep_conv)

    l2a = tf.nn.relu(tf.nn.conv2d(l1, w2,                     # l2a shape=(?, 14, 14, 64)
                        strides=[1, 1, 1, 1], padding='SAME'))
    l2 = tf.nn.max_pool(l2a, ksize=[1, 2, 2, 1],              # l2 shape=(?, 7, 7, 64)
                        strides=[1, 2, 2, 1], padding='SAME')
    l2 = tf.nn.dropout(l2, p_keep_conv)

    l3a = tf.nn.relu(tf.nn.conv2d(l2, w3,                     # l3a shape=(?, 7, 7, 128)
                        strides=[1, 1, 1, 1], padding='SAME'))
    l3 = tf.nn.max_pool(l3a, ksize=[1, 2, 2, 1],              # l3 shape=(?, 4, 4, 128)
                        strides=[1, 2, 2, 1], padding='SAME')
    l3 = tf.reshape(l3, [-1, w4.get_shape().as_list()[0]])    # reshape to (?, 2048)
    l3 = tf.nn.dropout(l3, p_keep_conv)

    l4 = tf.nn.relu(tf.matmul(l3, w4))
    l4 = tf.nn.dropout(l4, p_keep_hidden)

    pyx = tf.matmul(l4, w_o)
    return pyx

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels
trX = trX.reshape(-1, 28, 28, 1)  # 28x28x1 input img
teX = teX.reshape(-1, 28, 28, 1)  # 28x28x1 input img

In [0]:
X = tf.placeholder("float", [None, 28, 28, 1])
Y = tf.placeholder("float", [None, 10])

w = init_weights([3, 3, 1, 32])       # 3x3x1 conv, 32 outputs, 32개의 필터
w2 = init_weights([3, 3, 32, 64])     # 3x3x32 conv, 64 outputs
w3 = init_weights([3, 3, 64, 128])    # 3x3x32 conv, 128 outputs
w4 = init_weights([128 * 4 * 4, 625]) # FC 128 * 4 * 4 inputs, 625 outputs
w_o = init_weights([625, 10])         # FC 625 inputs, 10 outputs (labels)

p_keep_conv = tf.placeholder("float")
p_keep_hidden = tf.placeholder("float")
py_x = model(X, w, w2, w3, w4, w_o, p_keep_conv, p_keep_hidden)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=py_x, labels=Y))
train_op = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)
predict_op = tf.argmax(py_x, 1)

In [0]:
# Launch the graph in a session
with tf.Session() as sess:
    # you need to initialize all variables
    tf.global_variables_initializer().run()

    for i in range(100):
        training_batch = zip(range(0, len(trX), batch_size),
                             range(batch_size, len(trX)+1, batch_size))
        for start, end in training_batch:
            sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end],
                                          p_keep_conv: 0.8, p_keep_hidden: 0.5})

        test_indices = np.arange(len(teX)) # Get A Test Batch
        np.random.shuffle(test_indices)
        test_indices = test_indices[0:test_size]

        print(i, np.mean(np.argmax(teY[test_indices], axis=1) ==
                         sess.run(predict_op, feed_dict={X: teX[test_indices],
                                                         Y: teY[test_indices],
                                                         p_keep_conv: 1.0,
                                                         p_keep_hidden: 1.0})))

* https://www.coursera.org/learn/convolutional-neural-networks/
* https://predictiveprogrammer.com/famous-convolutional-neural-network-architectures-1/
* https://predictiveprogrammer.com/famous-convolutional-neural-network-architectures-2/
* https://towardsdatascience.com/applied-deep-learning-part-4-convolutional-neural-networks-584bc134c1e2
* 


##  6.4 feature map의 시각화
https://cs.stanford.edu/people/karpathy/convnetjs/demo/cifar10.html